In [10]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import pickle
from datetime import datetime

In [2]:
#read sql db
engine = create_engine("sqlite:///sold_houses.db")

In [3]:
engine.table_names()

<ipython-input-3-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['houses']

In [4]:
df = pd.read_sql('SELECT * FROM houses;', engine)

In [5]:
df.columns

Index(['index', 'sale_type', 'sold_date', 'property_type', 'address', 'city',
       'state', 'zip_code', 'sold_price', 'beds', 'baths', 'location', 'sqft',
       'lot_size', 'year_built', 'days_on_market', 'price_sqft', 'HOA',
       'status', 'open_house_start', 'open_house_end', 'url', 'source', 'MLS',
       'favorite', 'interested', 'latitude', 'longitude'],
      dtype='object')

In [6]:
#drop values where no price is available
df =df[df['sold_price'].notna()]
df.shape

(110923, 28)

In [7]:
#Impute missing values for sold date
df['sold_date'] = df['sold_date']. replace('nan', np.nan)
df['sold_date'] = df['sold_date'].fillna(method='ffill')

In [8]:
def new_const(row):
    if row['year_built'] >= 2021:
        return 1
    else:
        return 0  
df['new_const'] = df.apply(new_const, axis=1)

In [9]:
df = df.drop('index', 1)

In [11]:
#convert to datetime
df['sold_date'] = df['sold_date'].astype(str)
def convert_date(row):
    return datetime.strptime(row["sold_date"], "%Y-%m-%d")
houses_data['sold_date'] = houses_data.apply(convert_date, axis=1)

NameError: name 'houses_data' is not defined

In [82]:
df.to_csv("houses_data.csv")